




























Append row data to DataFrame: The row data (dictionary) obtained in the previous step is appended to the DataFrame df using df.append(row_data, ignore_index=True).

Display the DataFrame: Finally, the populated DataFrame is displayed, showing the data extracted from the table on the webpage.

1. Import necessary libraries: The code begins by importing the required libraries, webdriver from Selenium and BeautifulSoup from bs4. These libraries are used for web scraping and parsing the HTML content of a webpage.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

2. Set the URL: The URL of the webpage to scrape is assigned to the variable url.

In [ ]:
url = "https://dps.psx.com.pk/listings"

3.Initialize Chrome WebDriver: A Chrome WebDriver is initialized using webdriver.Chrome(). This allows Selenium to control the Chrome browser programmatically.

In [ ]:
driver = webdriver.Chrome()

4. Load the webpage: The browser is directed to open the specified URL using driver.get(url).

In [ ]:
driver.get(url)

5.Implicitly wait for content to load: An implicit wait of 10 seconds is set using driver.implicitly_wait(10). This means the WebDriver will wait for up to 10 seconds for the dynamic content to load before proceeding to the next step. The wait time can be adjusted based on the website's loading speed

In [ ]:
driver.implicitly_wait(5)

6. In the given code, <html = driver.page_source>, it retrieves the full HTML source code of the currently loaded web page using Selenium WebDriver and assigns it to the variable "html.

In [ ]:
html = driver.page_source

7. Close the WebDriver: The WebDriver is closed to release system resources using driver.quit().

In [ ]:
driver.quit()

8. Now use BeautifulSoup to parse the page source

In [ ]:
soup = BeautifulSoup(html,"lxml")

9.Find the table: The code finds the table element in the HTML using the find() method of BeautifulSoup. It looks for the table with the class "tbl dataTable no-footer" and assigns it to the variable table.

In [ ]:
table = soup.find("table",class_ ="tbl dataTable no-footer")

10. Find the table header: Using the find() method again, the code searches for the table header (thead) within the previously found table.

In [ ]:
headers = table.find("thead",class_="tbl__head")

11. Extract column names (headers): The code extracts the column names (header titles) from the **`th elements`** inside the header. It creates an empty list titles and loops through the header cells, extracting the text from each cell and adding it to the titles list.

In [ ]:
# Create a empty list 
titles = []
#if headers: line is to check whether the headers variable has some content before proceeding with the code inside the if-block.
if headers:
    # The code extracts the column names (header titles) from the **`th elements`**
    header_cells = headers.find_all('th')
    for header in header_cells:
        header = header.text.strip()
        titles.append(header)
print(titles)

11. Create an empty DataFrame: A pandas DataFrame named df is created with columns specified by the extracted titles.

In [ ]:
df = pd.DataFrame(columns= titles)

12. Find the table body: Using the find() method, the code searches for the table body (tbody) within the previously found table.

13. Find all rows: The code uses find_all() method on the tbody to find all rows `(tr)` table row present in the table.

In [ ]:
#Step 7: Create an empty DataFrame with the extracted header titles
df = pd.DataFrame(columns=titles)

#Step 8: Find the table body (tbody) containing the data rows
body = table.find('tbody')
#Step 9: Find all rows (tr) in the table body
rows = body.find_all('tr')


13.Loop through rows and extract data: The code iterates through each row found in the previous step and extracts the data from the individual cells 
**`(td)`** is called table data. The text content of each cell is stripped of any leading or trailing spaces and stored in the data list.

In [ ]:
for row in rows:
    data_cells = row.find_all('td')
    data  = [cell.text.strip() for cell in data_cells]
    row_data = dict(zip(titles,data))
    df = df.append(row_data,ignore_index=True)

In [ ]:
df.shape

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

url = "https://dps.psx.com.pk/listings"

# Use Chrome WebDriver (make sure chromedriver executable is in the same directory)
driver = webdriver.Chrome()

driver.get(url)

# Wait for the dynamic content to load (you may need to adjust the wait time based on the website's loading speed)
driver.implicitly_wait(10)

# Get the page source after JavaScript has rendered the table
html = driver.page_source

# Close the WebDriver
driver.quit()

# Now use BeautifulSoup to parse the page source
soup = BeautifulSoup(html, "lxml")

table = soup.find('table', class_="tbl dataTable no-footer")
#DataTables_Table_0
# Step 5: Find the table header (thead) to extract the column names
headers = table.find('thead', class_="tbl__head")

# Step 6: Get the header titles by finding all the `th` elements in the header
titles = []
if headers:
    header_cells = headers.find_all('th')
    for header in header_cells:
        title = header.text.strip()
        titles.append(title)

#Step 7: Create an empty DataFrame with the extracted header titles
df = pd.DataFrame(columns=titles)

#Step 8: Find the table body (tbody) containing the data rows
body = table.find('tbody')

#Step 9: Find all rows (tr) in the table body
rows = body.find_all('tr')

#Step 10: Loop through the rows and extract data from the cells (td)
for row in rows:
    data_cells = row.find_all('td')
    data = [cell.text.strip() for cell in data_cells]
    
#Step 11: Create a dictionary with column names as keys and row data as values
    row_data = dict(zip(titles, data))
    
#Step 12: Append the row data to the DataFrame
    df = df.append(row_data, ignore_index=True)
#Step 13: Display the populated DataFrame

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_all_pages(base_url):
    driver = webdriver.Chrome()
    driver.get(base_url)
    driver.implicitly_wait(10)

    # Get the initial page source
    html = driver.page_source
    all_data = pd.DataFrame()

    while True:
        soup = BeautifulSoup(html, "lxml")
        tables = soup.find_all('table', class_='tbl dataTable no-footer')

        for table in tables:
            headers = table.find('thead', class_='tbl__head')
            titles = [header.text.strip() for header in headers.find_all('th')]

            body = table.find('tbody')
            rows = body.find_all('tr')

            for row in rows:
                data_cells = row.find_all('td')
                data = [cell.text.strip() for cell in data_cells]
                row_data = dict(zip(titles, data))
                all_data = all_data.append(row_data, ignore_index=True)

        # Check if there is a "Next" button, and click if it exists
        next_button = driver.find_element(By.XPATH, "//a[contains(@class, 'paginate_button next')]")
        if "disabled" in next_button.get_attribute("class"):
            # Assume we have reached the last page, stop scraping
            break

        next_button.click()
        time.sleep(3)  # Add a short delay to allow content to load
        html = driver.page_source

    driver.quit()
    return all_data

if __name__ == "__main__":
    base_url = "https://dps.psx.com.pk/listings"
    result_df = scrape_all_pages(base_url)
    print(result_df.shape)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_8984\136646913.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(row_data, ignore_index=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_8984\136646913.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(row_data, ignore_index=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_8984\136646913.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(row_data, ignore_index=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_8984\136646913.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.appe

(452, 7)


In [13]:
result_df.shape

(452, 7)

In [16]:
result_df.tail(200)


,Symbol,Name,Sector,Clearing Type,Shares,Free Float,Listed In
252,LPL,Lalpir Power Ltd.,POWER GENERATION & DISTRIBUTION,NC,"379,838,732","151,935,493",ALLSHRKMIALLSHRPSXDIV20
253,LSEPL,LSE Proptech Ltd.,TECHNOLOGY & COMMUNICATION,NC,"90,090,731","49,516,717",ALLSHR
254,LSEVL,LSE Ventures Limited,INV. BANKS / INV. COS. / SECURITIES COS.,NC,"179,597,880","116,738,612",ALLSHR
255,LUCK,Lucky Cement Ltd,CEMENT,NC,"313,375,000","109,681,250",ACIALLSHRKMI30KMIALLSHRKSE100KSE30MZNPINBPPGIN...
256,MACFL,MACPAC Films Ltd.,MISCELLANEOUS,NC,"59,301,150","20,755,403",ALLSHRKMIALLSHR
...,...,...,...,...,...,...,...
447,WTL,WorldCall Telecom Ltd,TECHNOLOGY & COMMUNICATION,NC,"3,196,199,269","3,109,831,057",ALLSHRKMIALLSHR
448,YOUW,Yousaf Weaving Mills Ltd,TEXTILE WEAVING,NC,"90,000,000","49,500,000",ALLSHRKSE100
449,ZAHID,Zahidjee Textile Mills Ltd.,TEXTILE COMPOSITE,NC,"191,421,102","5,912,547",ALLSHRKMIALLSHR
450,ZIL,ZIL Limited.,FOOD & PERSONAL CARE PRODUCTS,NC,"6,122,600","306,130",ALLSHRKMIALLSHR


In [ ]:
result_df.to_csv("Pakistan_Stocks_Exchange_lisited_Companies(2023).csv")